In [47]:
import warnings
import os
from IPython.display import Audio, display
import numpy as np
import librosa
from librosa.display import specshow
import pandas as pd
import matplotlib.pyplot as plt
from scipy.fftpack import fft, dct, ifft
from sklearn.model_selection import train_test_split
from scipy.stats import norm
warnings.simplefilter('ignore')
%matplotlib inline

In [ ]:
import pywt

In [4]:
train_fl = pd.read_csv('./train_files.csv', header=None)
test_fl = pd.read_csv('./test_files.csv', header=None)
valid_fl = pd.read_csv('./val_files.csv', header=None)

In [5]:
data = pd.read_csv('./data/list_mfccs.csv')

In [6]:
import sklearn

In [49]:
from sklearn.mixture import GaussianMixture
from sklearn import preprocessing

In [8]:
cols = ['mfcc_%s'%i for i in range(1, 40)]
scaler = preprocessing.StandardScaler()
scaled_df = scaler.fit_transform(data[cols])


In [9]:
data_scaled = data.copy()
data_scaled[cols] = pd.DataFrame(scaled_df)

In [10]:
train = data_scaled[data.filename.map(lambda x: x in set(train_fl[1]))]
test = data_scaled[data.filename.map(lambda x: x in set(test_fl[1]))]
val = data_scaled[data.filename.map(lambda x: x in set(valid_fl[1]))]

In [48]:
train['idx'] = list(train.index)
test['idx'] = list(test.index)
val['idx'] = list(val.index)

In [22]:
train.head()

,filename,class,lbl,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,...,mfcc_31,mfcc_32,mfcc_33,mfcc_34,mfcc_35,mfcc_36,mfcc_37,mfcc_38,mfcc_39,idx
0,blues/blues.00000.au,blues,1,-0.724054,-0.500629,-0.165197,-0.224717,0.344881,0.522743,-1.350355,...,0.028203,-0.075622,-0.863715,-1.345201,-0.072421,0.18751,0.406762,0.196529,-0.217599,0
1,blues/blues.00000.au,blues,1,-0.525030,-0.067544,-0.145980,0.647703,0.637306,0.695705,-0.702556,...,0.028203,-0.075622,-0.863715,-1.345201,-0.072421,0.18751,0.268448,0.196529,-0.217599,1
2,blues/blues.00000.au,blues,1,-0.590512,-0.206208,-0.388126,0.741486,0.722044,0.497295,-0.975393,...,0.028203,-0.075622,-0.863715,-1.345201,-0.072421,0.18751,0.325000,0.196529,-0.217599,2
3,blues/blues.00000.au,blues,1,-0.498473,-0.023238,-0.278193,0.395156,-0.000983,0.510387,-1.066385,...,0.028203,-0.075622,-0.863715,-1.345201,-0.072421,0.18751,0.239851,0.196529,-0.217599,3
4,blues/blues.00000.au,blues,1,-0.456160,-0.403885,-0.030121,1.040700,0.450166,1.400972,0.105636,...,0.028203,-0.075622,-0.863715,-1.345201,-0.072421,0.18751,0.191897,0.196529,-0.217599,4


In [110]:
pd.DataFrame(train.groupby('class').size())

,0
class,
blues,160000
classical,160000
country,160000
disco,160000
hiphop,160000
jazz,160000
metal,160000
pop,160000
reggae,80000


In [29]:
classes = set( ['classical', 'jazz', 'metal'])

In [34]:
tmp_data = data[data['class'].map(lambda x: x in classes)]

In [111]:
tmp_train = train[train['class'].map(lambda x: x in classes)]
tmp_test = test[test['class'].map(lambda x: x in classes)]
tmp_val = val[val['class'].map(lambda x: x in classes)]

In [114]:
gmm = GaussianMixture(n_components=200, random_state=45, verbose=1)

In [ ]:
%%time
gmm.fit(tmp_train[cols].as_matrix())

Initialization 0
  Iteration 0
  Iteration 10
  Iteration 20


In [55]:
pred_train = gmm.predict_proba(tmp_train[cols].as_matrix())

In [56]:
pred_val = gmm.predict_proba(tmp_val[cols].as_matrix())

In [65]:
mn = gmm.means_

In [ ]:
... цикл по трекам...
M = []
for i in range(train.shape[0] // 2000):
    tmp = (pred_train[i*2000:(i+1)*2000, :, None]
           * tmp_train[cols].as_matrix()[i*2000:(i+1)*2000, None, :]
          ).sum(axis=0)
    tmp = tmp / pred_train[i*2000:(i+1)*2000].sum(axis=0)[:, None]
    M.append(tmp.reshape(1, -1))

In [64]:
M = np.vstack(M)

In [75]:
from sklearn.decomposition import FactorAnalysis

In [101]:
fa = FactorAnalysis(n_components=400)

In [97]:
%%%time
fa.fit(M)

Wall time: 119 ms


FactorAnalysis(copy=True, iterated_power=3, max_iter=1000, n_components=400,
        noise_variance_init=None, random_state=0, svd_method='randomized',
        tol=0.01)

In [98]:
m0 = fa.fit_transform(M.T)

In [100]:
M.T.shape

(39, 500)